<a href="https://colab.research.google.com/github/aicuai/GenAI-Steam/blob/main/20250314_GeminiAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-genai --break-system-packages


In [ ]:
from google import genai
from google.genai import types
from PIL import Image
from IPython.display import display
from io import BytesIO
from google.colab import userdata
import time

client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))
contents = ('Hi, can you create an image of two boys under the sakura tree?')
print(contents)

while True:
    response = client.models.generate_content(
        model="models/gemini-2.0-flash-exp",
        contents=contents,
        config=types.GenerateContentConfig(response_modalities=['Text', 'Image'])
    )

    if response.candidates[0].content is not None:
        for part in response.candidates[0].content.parts:
            if part.text is not None:
                print(part.text)
            elif part.inline_data is not None:
                image = Image.open(BytesIO(part.inline_data.data))
                display(image)
                image.save('temp_image.png')
        break  # コンテンツが返されたらループを抜ける
    else:
        print("エラー: Geminiモデルが期待する形式でコンテンツを返していません。待機して再試行します。")
        time.sleep(60)  # 待機

In [ ]:
from google import genai
from google.genai import types
from PIL import Image
from IPython.display import display
from io import BytesIO
from google.colab import userdata
import base64

client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

def generate_image(prompt, image_path='tmp_img.png', context=""): # contextを追加
    # contextをpromptに追加
    full_prompt = f"{context}\n{prompt}"

    response = client.models.generate_content(
        model="models/gemini-2.0-flash-exp",
        contents=full_prompt,  # full_promptを使用
        config=types.GenerateContentConfig(response_modalities=['Text', 'Image'])
    )

    image = None
    for part in response.candidates[0].content.parts:
        if part.text is not None:
            print(part.text)
        elif part.inline_data is not None:
            image = Image.open(BytesIO(part.inline_data.data))
            break  # 画像が見つかったらループを抜ける

    if image:
        image.save(image_path)  # 画像を保存

    return image, full_prompt # full_promptを返す

def run_and_display(prompt, context=""): # contextを追加
    image, new_context = generate_image(prompt, context=context) # contextを渡す
    if image:
        display(image)
    return new_context # new_contextを返す



# 対話コマンド


In [ ]:
# 対話コマンド
# context = run_and_display("鶏のつがいのイラストレーションを生成して")  # 最初の画像生成
# context = run_and_display("背景を除去して", context=context)  # 前回生成された画像に対して背景除去
context = run_and_display("桜の下で約束をする高校生のイラストレーション")
context = run_and_display("ダイナミックな構図で背後下から", context=context)
context = run_and_display("白黒漫画風に", context=context)
